# Validación en Producción con Datos Actualizados

Este notebook:
1. Carga datos actualizados de los datasets originales (más recientes que los usados en entrenamiento)
2. Procesa y prepara los nuevos datos siguiendo el mismo pipeline
3. Genera predicciones usando el modelo ganador
4. Compara las predicciones con los datos reales actualizados
5. Evalúa el rendimiento del modelo en datos completamente nuevos

**Objetivo**: Validar que el modelo mantiene su rendimiento con datos fuera del período de entrenamiento y validación.

In [ ]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import json
import joblib
import holidays

# Sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

warnings.filterwarnings('ignore')

# Configuración
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
plt.style.use('default')
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [14, 6]

print("Notebook de Validación en Producción")
print("="*60)

## 1. Identificar Período de Datos Nuevos

Primero identificamos qué período se usó en entrenamiento/validación y qué datos nuevos tenemos disponibles.

In [ ]:
# Cargar datos de validación usados en el notebook 08
df_validation_anterior = pd.read_parquet('artifacts/data/train_models/features_validation.parquet')

print("Período usado en validación anterior:")
print(f"  Inicio: {df_validation_anterior['datetime'].min()}")
print(f"  Fin:    {df_validation_anterior['datetime'].max()}")
print(f"  Registros: {len(df_validation_anterior):,}")

# Definir fecha de inicio para datos nuevos (después del último dato de validación)
FECHA_INICIO_NUEVOS = df_validation_anterior['datetime'].max() + pd.Timedelta(hours=1)

print(f"\nBuscando datos nuevos desde: {FECHA_INICIO_NUEVOS}")

## 2. Cargar Datos Actualizados de Fuentes Originales

Cargamos los datos de las mismas fuentes que se usaron en el notebook 01.

In [ ]:
# Cargar datos de validación usados en el notebook 09
validation_data_path = Path('artifacts/data/validation_models/features_validation.parquet')
if validation_data_path.exists():
    df_validation_original = pd.read_parquet(validation_data_path)
    validation_end = df_validation_original['datetime'].max()
    print(f"✓ Datos de validación cargados (NB09)")
    print(f"  Período de validación terminó el: {validation_end}")
else:
 print("ADVERTENCIA: No se encontraron datos de validación del NB09")
    validation_end = None

In [ ]:
# Cargar datos meteorológicos
print("\nCargando datos meteorológicos...")
meteo_files = list(Path('../../data/climatologia/data_parquet_clean/meteo').glob('*.parquet'))
if meteo_files:
    df_meteo_full = pd.concat([pd.read_parquet(f) for f in meteo_files], ignore_index=True)
    df_meteo_full['datetime'] = pd.to_datetime(df_meteo_full['datetime'])
    df_meteo_full = df_meteo_full.sort_values('datetime').drop_duplicates(subset=['datetime'])
    print(f"  ✓ Cargados {len(df_meteo_full):,} registros")
    print(f"  Período: {df_meteo_full['datetime'].min()} a {df_meteo_full['datetime'].max()}")
    print(f"  Columnas: {len(df_meteo_full.columns)}")
else:
    print("  ✗ No se encontraron archivos meteorológicos")
    df_meteo_full = None

In [ ]:
# Cargar datos de precios
print("\nCargando datos de precios...")
precio_files = list(Path('../../data/precio_luz/data_parquet_clean').glob('*.parquet'))
if precio_files:
    df_precios_full = pd.concat([pd.read_parquet(f) for f in precio_files], ignore_index=True)
    df_precios_full['datetime'] = pd.to_datetime(df_precios_full['datetime'])
    df_precios_full = df_precios_full.sort_values('datetime').drop_duplicates(subset=['datetime'])
    print(f"  ✓ Cargados {len(df_precios_full):,} registros")
    print(f"  Período: {df_precios_full['datetime'].min()} a {df_precios_full['datetime'].max()}")
else:
    print("  ✗ No se encontraron archivos de precios")
    df_precios_full = None

In [ ]:
# Filtrar solo datos nuevos (posteriores a la validación anterior)
print("\n" + "="*60)
print("FILTRANDO DATOS NUEVOS")
print("="*60)

if df_demanda_full is not None:
    df_demanda_nuevos = df_demanda_full[df_demanda_full['datetime'] >= FECHA_INICIO_NUEVOS].copy()
    print(f"\nDemanda - Datos nuevos: {len(df_demanda_nuevos):,} registros")
    if len(df_demanda_nuevos) > 0:
        print(f"  Desde: {df_demanda_nuevos['datetime'].min()}")
        print(f"  Hasta: {df_demanda_nuevos['datetime'].max()}")
    else:
        print("  No hay datos nuevos de demanda")

if df_meteo_full is not None:
    df_meteo_nuevos = df_meteo_full[df_meteo_full['datetime'] >= FECHA_INICIO_NUEVOS].copy()
    print(f"\nMeteo - Datos nuevos: {len(df_meteo_nuevos):,} registros")
    if len(df_meteo_nuevos) > 0:
        print(f"  Desde: {df_meteo_nuevos['datetime'].min()}")
        print(f"  Hasta: {df_meteo_nuevos['datetime'].max()}")

if df_precios_full is not None:
    df_precios_nuevos = df_precios_full[df_precios_full['datetime'] >= FECHA_INICIO_NUEVOS].copy()
    print(f"\nPrecios - Datos nuevos: {len(df_precios_nuevos):,} registros")
    if len(df_precios_nuevos) > 0:
        print(f"  Desde: {df_precios_nuevos['datetime'].min()}")
        print(f"  Hasta: {df_precios_nuevos['datetime'].max()}")

## 3. Procesar Datos Nuevos

Aplicamos el mismo procesamiento que en el notebook 01 y 02.

In [ ]:
def crear_caracteristicas_temporales(df):
    """
    Crea características temporales basadas en la columna de datetime.
    """
    df = df.copy()
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Características cíclicas
    df['hora_del_dia_sin'] = np.sin(2 * np.pi * df['datetime'].dt.hour/24)
    df['hora_del_dia_cos'] = np.cos(2 * np.pi * df['datetime'].dt.hour/24)
    df['dia_semana_sin'] = np.sin(2 * np.pi * df['datetime'].dt.dayofweek/7)
    df['dia_semana_cos'] = np.cos(2 * np.pi * df['datetime'].dt.dayofweek/7)
    df['mes_sin'] = np.sin(2 * np.pi * df['datetime'].dt.month/12)
    df['mes_cos'] = np.cos(2 * np.pi * df['datetime'].dt.month/12)

    # Características categóricas
    df['hora_del_dia'] = df['datetime'].dt.hour
    df['dia_semana'] = df['datetime'].dt.dayofweek
    df['mes'] = df['datetime'].dt.month
    df['trimestre'] = df['datetime'].dt.quarter
    df['año'] = df['datetime'].dt.year
    df['es_festivo'] = df['datetime'].dt.date.isin(
        holidays.Spain(years=df['datetime'].dt.year.unique().tolist())
    ).astype(int)

    # Características binarias
    df['es_finde'] = df['dia_semana'].isin([5, 6]).astype(int)
    df['es_laboral'] = (~df['dia_semana'].isin([5, 6])).astype(int)
    df['es_hora_pico_mañana'] = df['hora_del_dia'].between(7, 9).astype(int)
    df['es_hora_pico_tarde'] = df['hora_del_dia'].between(18, 20).astype(int)
    
    return df

def crear_lags_y_medias_moviles(df, columna_target='real', lags=[1, 24, 48, 168]):
    """
    Crea características de valores retardados y medias móviles.
    """
    df = df.copy()
    
    # Valores retardados
    for lag in lags:
        df[f'lag_{lag}h'] = df[columna_target].shift(lag)
    
    # Medias móviles
    ventanas = [6, 12, 24, 48, 168]
    for ventana in ventanas:
        df[f'media_movil_{ventana}h'] = df[columna_target].rolling(window=ventana).mean()
        df[f'std_movil_{ventana}h'] = df[columna_target].rolling(window=ventana).std()
        
    return df

In [ ]:
# Necesitamos datos históricos para calcular lags correctamente
# Combinamos los últimos 7 días de los datos anteriores con los datos nuevos
print("Preparando datos para calcular lags...")

fecha_inicio_contexto = FECHA_INICIO_NUEVOS - pd.Timedelta(days=7)
df_demanda_contexto = df_demanda_full[
    df_demanda_full['datetime'] >= fecha_inicio_contexto
].copy()

print(f"  Datos con contexto: {len(df_demanda_contexto):,} registros")
print(f"  Desde: {df_demanda_contexto['datetime'].min()}")
print(f"  Hasta: {df_demanda_contexto['datetime'].max()}")

In [ ]:
# Crear características temporales
print("\nCreando características temporales...")
df_demanda_procesado = crear_caracteristicas_temporales(df_demanda_contexto)
print(f"  ✓ Características temporales creadas")

# Crear lags y medias móviles
print("\nCreando lags y medias móviles...")
df_demanda_procesado = crear_lags_y_medias_moviles(df_demanda_procesado, columna_target='real')
print(f"  ✓ Lags y medias móviles creadas")

# Renombrar columna 'real' a 'demanda' para consistencia
df_demanda_procesado = df_demanda_procesado.rename(columns={'real': 'demanda'})

print(f"\nDatos procesados: {df_demanda_procesado.shape}")
print(f"Columnas: {len(df_demanda_procesado.columns)}")

## 4. Agregar Datos Meteorológicos

Agregamos los datos meteorológicos usando los mismos pesos de población que en entrenamiento.

In [ ]:
# Pesos de población (los mismos que en el entrenamiento)
pesos_ciudades = {
    'albacete': 0.008943,
    'alicante': 0.045363,
    'almeria': 0.016711,
    'avila': 0.003591,
    'badajoz': 0.015198,
    'barcelona': 0.133783,
    'bilbao': 0.026242,
    'burgos': 0.008017,
    'caceres': 0.008784,
    'cadiz': 0.028428,
    'castellon': 0.013640,
    'ciudad_real': 0.011110,
    'cordoba': 0.017591,
    'a_coruna': 0.025292,
    'cuenca': 0.004426,
    'girona': 0.017908,
    'granada': 0.021231,
    'guadalajara': 0.006007,
    'huelva': 0.012194,
    'huesca': 0.005036,
    'jaen': 0.014028,
    'leon': 0.010183,
    'lleida': 0.010002,
    'logrono': 0.007207,
    'lugo': 0.007452,
    'madrid': 0.159364,
    'malaga': 0.040151,
    'murcia': 0.035573,
    'ourense': 0.006888,
    'oviedo': 0.022764,
    'palencia': 0.003545,
    'pontevedra': 0.021340,
    'salamanca': 0.007430,
    'san_sebastian': 0.016259,
    'santander': 0.013323,
    'segovia': 0.003478,
    'sevilla': 0.044466,
    'soria': 0.001987,
    'tarragona': 0.019308,
    'teruel': 0.003026,
    'toledo': 0.016101,
    'valencia': 0.061656,
    'valladolid': 0.011698,
    'vitoria': 0.007520,
    'zamora': 0.003794,
    'zaragoza': 0.021950
}

def agregar_datos_meteo(weather_df, pesos_ciudades):
    """
    Agrega los datos meteorológicos usando una media ponderada por población.
    """
    # Obtener las columnas base (sin sufijos de ciudad)
    cols_base = ['temperature_2m', 'precipitation', 'cloud_cover', 'wind_speed_10m']
    
    # Normalizar pesos para que sumen 1
    total = sum(pesos_ciudades.values())
    pesos_norm = {k: v/total for k, v in pesos_ciudades.items()}
    
    # Inicializar DataFrame agregado
    weather_agg = pd.DataFrame()
    weather_agg['datetime'] = weather_df['datetime'].unique()
    
    # Para cada variable meteorológica base
    for col_base in cols_base:
        weighted_sum = pd.Series(0, index=weather_agg.index)
        total_weight = 0
        
        # Sumar las contribuciones ponderadas de cada ciudad
        for ciudad, peso in pesos_norm.items():
            col_ciudad = f"{col_base}_{ciudad}"
            if col_ciudad in weather_df.columns:
                weighted_sum += weather_df[col_ciudad].values * peso
                total_weight += peso
        
        # Normalizar por el peso total efectivo
        if total_weight > 0:
            weather_agg[col_base] = weighted_sum / total_weight
    
    return weather_agg

# Filtrar datos meteorológicos con contexto
df_meteo_contexto = df_meteo_full[
    df_meteo_full['datetime'] >= fecha_inicio_contexto
].copy()

# Agregar datos meteorológicos
print("Agregando datos meteorológicos...")
df_meteo_agg = agregar_datos_meteo(df_meteo_contexto, pesos_ciudades)
print(f"  ✓ Datos meteorológicos agregados: {df_meteo_agg.shape}")

## 5. Combinar Todos los Datos

In [ ]:
# Filtrar datos de precios con contexto
df_precios_contexto = df_precios_full[
    df_precios_full['datetime'] >= fecha_inicio_contexto
].copy()

print("Combinando todos los datos...")

# Merge con meteo
df_completo = df_demanda_procesado.merge(df_meteo_agg, on='datetime', how='left')
print(f"  Después de merge con meteo: {df_completo.shape}")

# Merge con precios
df_completo = df_completo.merge(df_precios_contexto, on='datetime', how='left')
print(f"  Después de merge con precios: {df_completo.shape}")

# Interpolar valores faltantes
print("\nInterpolando valores faltantes...")
n_nan_antes = df_completo.isna().sum().sum()
df_completo = df_completo.interpolate(method='linear')
n_nan_despues = df_completo.isna().sum().sum()
print(f"  NaN antes: {n_nan_antes}, después: {n_nan_despues}")

print(f"\nDataset completo: {df_completo.shape}")
print(f"Columnas: {len(df_completo.columns)}")

In [ ]:
# Filtrar solo los datos nuevos (sin el contexto histórico)
df_nuevos = df_completo[df_completo['datetime'] >= FECHA_INICIO_NUEVOS].copy()

# Eliminar filas con NaN
df_nuevos = df_nuevos.dropna()

print("\n" + "="*60)
print("DATOS NUEVOS LISTOS PARA VALIDACIÓN")
print("="*60)
print(f"Registros: {len(df_nuevos):,}")
print(f"Período: {df_nuevos['datetime'].min()} a {df_nuevos['datetime'].max()}")
print(f"Columnas: {len(df_nuevos.columns)}")
print("\nPrimeros registros:")
display(df_nuevos.head())
print("\nÚltimos registros:")
display(df_nuevos.tail())

## 6. Cargar Modelo Ganador y Hacer Predicciones

In [ ]:
# Cargar recomendación del modelo
with open('artifacts/analysis/model_recommendation.json', 'r') as f:
    recommendation = json.load(f)

modelo_ganador = recommendation['best_model']
print(f"Modelo ganador: {modelo_ganador}")
print(f"\nMétricas en validación anterior:")
for metric, value in recommendation['metrics'].items():
    if isinstance(value, float):
        print(f"  {metric}: {value:,.2f}" if metric != 'r2' else f"  {metric}: {value:.4f}")
    else:
        print(f"  {metric}: {value}")

# Cargar el pipeline del modelo
model_path = Path('artifacts/trained_models') / recommendation['model_file']
print(f"\nCargando modelo desde: {model_path}")
pipeline = joblib.load(model_path)
print("✓ Modelo cargado correctamente")

In [ ]:
# Preparar datos para predicción
if hasattr(pipeline, 'feature_names_in_'):
    required_features = pipeline.feature_names_in_
    print(f"Features requeridas por el modelo: {len(required_features)}")
    
    # Verificar que tenemos todas las features
    missing_features = set(required_features) - set(df_nuevos.columns)
    if missing_features:
        print(f"\nFeatures faltantes: {missing_features}")
        print("\nNo se pueden hacer predicciones sin estas features.")
    else:
        print("✓ Todas las features requeridas están disponibles")
        
        # Preparar X e y
        X_nuevos = df_nuevos[required_features].copy()
        y_real = df_nuevos['demanda'].copy()
        datetime_index = df_nuevos['datetime'].copy()
        
        print(f"\nDatos preparados:")
        print(f"  X shape: {X_nuevos.shape}")
        print(f"  y shape: {y_real.shape}")
else:
    print("El modelo no tiene feature_names_in_")
    required_features = [col for col in df_nuevos.columns if col not in ['datetime', 'demanda']]
    X_nuevos = df_nuevos[required_features].copy()
    y_real = df_nuevos['demanda'].copy()
    datetime_index = df_nuevos['datetime'].copy()

In [ ]:
# Hacer predicciones
print("\nHaciendo predicciones...")
y_pred = pipeline.predict(X_nuevos)
print("✓ Predicciones completadas")

# Crear DataFrame con resultados
df_resultados = pd.DataFrame({
    'datetime': datetime_index.values,
    'demanda_real': y_real.values,
    'demanda_predicha': y_pred,
    'error': y_real.values - y_pred,
    'error_abs': np.abs(y_real.values - y_pred),
    'error_porcentual': np.abs((y_real.values - y_pred) / y_real.values) * 100
})

print("\nPrimeras predicciones:")
display(df_resultados.head(10))
print("\nÚltimas predicciones:")
display(df_resultados.tail(10))

## 7. Evaluar Rendimiento en Datos Nuevos

In [ ]:
# Calcular métricas
mae = mean_absolute_error(y_real, y_pred)
rmse = np.sqrt(mean_squared_error(y_real, y_pred))
mape = mean_absolute_percentage_error(y_real, y_pred) * 100
r2 = r2_score(y_real, y_pred)

print("="*70)
print(f"EVALUACIÓN EN DATOS NUEVOS (PRODUCCIÓN)")
print(f"Período: {df_resultados['datetime'].min()} a {df_resultados['datetime'].max()}")
print(f"Registros: {len(df_resultados):,}")
print("="*70)
print(f"MAE:  {mae:,.2f} MW")
print(f"RMSE: {rmse:,.2f} MW")
print(f"MAPE: {mape:.2f}%")
print(f"R²:   {r2:.4f}")
print("="*70)

# Comparar con métricas de validación anterior
print("\n" + "="*70)
print("COMPARACIÓN CON VALIDACIÓN ANTERIOR")
print("="*70)

mae_anterior = recommendation['metrics']['mae']
rmse_anterior = recommendation['metrics']['rmse']
mape_anterior = recommendation['metrics']['mape']
r2_anterior = recommendation['metrics']['r2']

print(f"\nMAE:")
print(f"  Validación anterior: {mae_anterior:,.2f} MW")
print(f"  Datos nuevos:        {mae:,.2f} MW")
print(f"  Diferencia:          {mae - mae_anterior:+,.2f} MW ({((mae - mae_anterior)/mae_anterior)*100:+.1f}%)")

print(f"\nRMSE:")
print(f"  Validación anterior: {rmse_anterior:,.2f} MW")
print(f"  Datos nuevos:        {rmse:,.2f} MW")
print(f"  Diferencia:          {rmse - rmse_anterior:+,.2f} MW ({((rmse - rmse_anterior)/rmse_anterior)*100:+.1f}%)")

print(f"\nMAPE:")
print(f"  Validación anterior: {mape_anterior:.2f}%")
print(f"  Datos nuevos:        {mape:.2f}%")
print(f"  Diferencia:          {mape - mape_anterior:+.2f}%")

print(f"\nR²:")
print(f"  Validación anterior: {r2_anterior:.4f}")
print(f"  Datos nuevos:        {r2:.4f}")
print(f"  Diferencia:          {r2 - r2_anterior:+.4f}")

print("\n" + "="*70)

# Determinar si el modelo mantiene su rendimiento
umbral_degradacion = 0.10  # 10% de degradación aceptable
degradacion_mae = abs((mae - mae_anterior) / mae_anterior)
degradacion_mape = abs((mape - mape_anterior) / mape_anterior)

if degradacion_mae < umbral_degradacion and degradacion_mape < umbral_degradacion:
    print("✓ El modelo MANTIENE su rendimiento en datos nuevos")
elif degradacion_mae < umbral_degradacion * 2 and degradacion_mape < umbral_degradacion * 2:
    print("El modelo tiene una LIGERA degradación en datos nuevos")
else:
    print("✗ El modelo tiene una DEGRADACIÓN SIGNIFICATIVA en datos nuevos")
    print("   Se recomienda reentrenar el modelo con datos más recientes")

print("="*70)

In [ ]:
# Estadísticas detalladas de errores
print("\nEstadísticas de errores:")
print("\nError absoluto (MW):")
print(df_resultados['error_abs'].describe())
print("\nError porcentual (%):")
print(df_resultados['error_porcentual'].describe())

## 8. Visualización de Resultados

In [ ]:
# Visualizar predicciones vs real - Período completo
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# Gráfico 1: Predicciones vs Real
axes[0].plot(df_resultados['datetime'], df_resultados['demanda_real'], 
             label='Demanda Real', linewidth=2, alpha=0.8)
axes[0].plot(df_resultados['datetime'], df_resultados['demanda_predicha'], 
             label='Demanda Predicha', linewidth=2, alpha=0.8, linestyle='--')
axes[0].set_title(f'Validación en Producción - {modelo_ganador}', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Demanda (MW)', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Error
axes[1].plot(df_resultados['datetime'], df_resultados['error'], 
             color='red', linewidth=1, alpha=0.7)
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1].fill_between(df_resultados['datetime'], df_resultados['error'], 
                       alpha=0.3, color='red')
axes[1].set_title('Error de Predicción', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Error (MW)', fontsize=12)
axes[1].grid(True, alpha=0.3)

# Gráfico 3: Error Porcentual
axes[2].plot(df_resultados['datetime'], df_resultados['error_porcentual'], 
             color='orange', linewidth=1, alpha=0.7)
axes[2].axhline(y=mape, color='red', linestyle='--', alpha=0.5, 
                label=f'MAPE medio: {mape:.2f}%')
axes[2].set_title('Error Porcentual', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Fecha', fontsize=12)
axes[2].set_ylabel('Error (%)', fontsize=12)
axes[2].legend(fontsize=10)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Visualizar primera semana en detalle (si hay suficientes datos)
if len(df_resultados) >= 168:
    primera_semana = df_resultados.head(168)
    
    fig, axes = plt.subplots(2, 1, figsize=(16, 10))
    
    # Predicciones vs Real
    axes[0].plot(primera_semana['datetime'], primera_semana['demanda_real'], 
                 label='Demanda Real', linewidth=2, marker='o', markersize=3)
    axes[0].plot(primera_semana['datetime'], primera_semana['demanda_predicha'], 
                 label='Demanda Predicha', linewidth=2, marker='s', markersize=3, linestyle='--')
    axes[0].set_title('Primera Semana - Detalle', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Demanda (MW)', fontsize=12)
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)
    axes[0].tick_params(axis='x', rotation=45)
    
    # Error
    axes[1].bar(primera_semana['datetime'], primera_semana['error'], 
                color=['red' if e < 0 else 'green' for e in primera_semana['error']], 
                alpha=0.6, width=0.03)
    axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
    axes[1].set_title('Error - Primera Semana', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Fecha', fontsize=12)
    axes[1].set_ylabel('Error (MW)', fontsize=12)
    axes[1].grid(True, alpha=0.3, axis='y')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print(f"No hay suficientes datos para mostrar una semana completa ({len(df_resultados)} horas disponibles)")

In [ ]:
# Distribución de errores y scatter plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma de errores
axes[0].hist(df_resultados['error'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0].axvline(x=df_resultados['error'].mean(), color='green', linestyle='--', 
                linewidth=2, label=f'Media: {df_resultados["error"].mean():.2f} MW')
axes[0].set_title('Distribución de Errores', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Error (MW)', fontsize=12)
axes[0].set_ylabel('Frecuencia', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3, axis='y')

# Scatter plot: Real vs Predicho
axes[1].scatter(df_resultados['demanda_real'], df_resultados['demanda_predicha'], 
                alpha=0.5, s=20, color='steelblue')
# Línea diagonal perfecta
min_val = min(df_resultados['demanda_real'].min(), df_resultados['demanda_predicha'].min())
max_val = max(df_resultados['demanda_real'].max(), df_resultados['demanda_predicha'].max())
axes[1].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Predicción Perfecta')
axes[1].set_title('Demanda Real vs Predicha', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Demanda Real (MW)', fontsize=12)
axes[1].set_ylabel('Demanda Predicha (MW)', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Análisis por Hora del Día

In [ ]:
# Agregar hora del día
df_resultados['hora'] = pd.to_datetime(df_resultados['datetime']).dt.hour

# Agrupar por hora
errores_por_hora = df_resultados.groupby('hora').agg({
    'error_abs': ['mean', 'std'],
    'error_porcentual': 'mean',
    'demanda_real': 'mean',
    'demanda_predicha': 'mean'
}).reset_index()

errores_por_hora.columns = ['hora', 'mae', 'std', 'mape', 'demanda_real_media', 'demanda_pred_media']

# Visualizar
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Error absoluto medio por hora con barras de error
axes[0].bar(errores_por_hora['hora'], errores_por_hora['mae'], 
            yerr=errores_por_hora['std'], capsize=5,
            color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_title('Error Absoluto Medio por Hora del Día', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Hora del Día', fontsize=12)
axes[0].set_ylabel('MAE (MW)', fontsize=12)
axes[0].set_xticks(range(24))
axes[0].grid(True, alpha=0.3, axis='y')

# Demanda media real vs predicha por hora
axes[1].plot(errores_por_hora['hora'], errores_por_hora['demanda_real_media'], 
             marker='o', linewidth=2, label='Real', markersize=8)
axes[1].plot(errores_por_hora['hora'], errores_por_hora['demanda_pred_media'], 
             marker='s', linewidth=2, label='Predicha', markersize=8, linestyle='--')
axes[1].set_title('Demanda Media por Hora del Día', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Hora del Día', fontsize=12)
axes[1].set_ylabel('Demanda Media (MW)', fontsize=12)
axes[1].set_xticks(range(24))
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nError medio por hora del día:")
display(errores_por_hora[['hora', 'mae', 'mape']].round(2))

## 10. Guardar Resultados

In [ ]:
# Crear directorio para resultados de producción
output_dir = Path('artifacts/production_validation')
output_dir.mkdir(parents=True, exist_ok=True)

# Timestamp para los archivos
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Guardar predicciones
predictions_file = output_dir / f'predictions_{timestamp}.parquet'
df_resultados.to_parquet(predictions_file, index=False)
print(f"✓ Predicciones guardadas en: {predictions_file}")

# Guardar métricas
metricas_produccion = {
    'timestamp': timestamp,
    'fecha_inicio': df_resultados['datetime'].min().isoformat(),
    'fecha_fin': df_resultados['datetime'].max().isoformat(),
    'n_registros': len(df_resultados),
    'modelo': modelo_ganador,
    'metricas_produccion': {
        'mae': float(mae),
        'rmse': float(rmse),
        'mape': float(mape),
        'r2': float(r2)
    },
    'metricas_validacion_anterior': recommendation['metrics'],
    'comparacion': {
        'mae_diff': float(mae - mae_anterior),
        'mae_diff_pct': float(((mae - mae_anterior) / mae_anterior) * 100),
        'rmse_diff': float(rmse - rmse_anterior),
        'rmse_diff_pct': float(((rmse - rmse_anterior) / rmse_anterior) * 100),
        'mape_diff': float(mape - mape_anterior),
        'r2_diff': float(r2 - r2_anterior)
    },
    'rendimiento': 'BUENO' if degradacion_mae < umbral_degradacion else 'DEGRADADO'
}

metrics_file = output_dir / f'metrics_{timestamp}.json'
with open(metrics_file, 'w') as f:
    json.dump(metricas_produccion, f, indent=2)
print(f"✓ Métricas guardadas en: {metrics_file}")

# Guardar análisis por hora
errores_hora_file = output_dir / f'errores_por_hora_{timestamp}.csv'
errores_por_hora.to_csv(errores_hora_file, index=False)
print(f"✓ Análisis por hora guardado en: {errores_hora_file}")

print("\n" + "="*70)
print("VALIDACIÓN EN PRODUCCIÓN COMPLETADA")
print("="*70)
print(f"Período validado: {df_resultados['datetime'].min()} a {df_resultados['datetime'].max()}")
print(f"Registros: {len(df_resultados):,}")
print(f"MAE: {mae:,.2f} MW (vs {mae_anterior:,.2f} MW en validación)")
print(f"MAPE: {mape:.2f}% (vs {mape_anterior:.2f}% en validación)")
print(f"\nResultados guardados en: {output_dir}")
print("="*70)